### 正则表达

In [1]:
import re
phone_reg = re.compile(r"(1[35678]\d{9})")
re.findall(phone_reg,'合作联系方式联系：18611696673 歌手，萨达')

['18611696673']

### 传统 NER

https://sklearn-crfsuite.readthedocs.io/en/latest/install.html


sklearn-crfsuite (sklearn) -> python-crfsuite (python) -> crfsuite (c)

In [18]:
import glob
import sklearn
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from  utils import utils
from imp import reload
reload(utils)

<module 'utils.utils' from '/Users/wanjun/Desktop/比赛训练营/DataFoutain隐私识别/DBC_code/src/utils/utils.py'>

In [ ]:
tdata_lst = glob('../data/train_data/*.txt')
symbol = ['。','!','！','?','？','﹗']
# 生成样本的参数
symbol = ['？','⋯','…','﹗']
max_sent_length = 250
max_input_length = 300

In [ ]:
from sklearn.model_selection import train_test_split
from glob import glob
data_lst = glob('../data/train_data/*.txt')
train_lst,val_lst = train_test_split(data_lst,test_size=0.1,random_state=0)
print('train:{}'.format(len(train_lst)))
print('val:{}'.format(len(val_lst)))

In [ ]:
data_dir = '../data/train_label'
for data_path in tqdm(train_lst):
    file_name = os.path.basename(data_path).split('.')[0]
    label_path = file_name+'.csv'
    label_path = os.path.join(data_dir,label_path)
    output_path = '../data/train_new_bioes'
    output_file = file_name
    utils.gen_data(label_path, data_path, output_path, output_file,symbol,max_sent_length,max_input_length,bioes = True)
for data_path in tqdm(val_lst):
    file_name = os.path.basename(data_path).split('.')[0]
    label_path = file_name+'.csv'
    label_path = os.path.join(data_dir,label_path)
    output_path = '../data/val_new_bioes'
    output_file = file_name
    utils.gen_data(label_path, data_path, output_path, output_file,symbol,max_sent_length,max_input_length,bioes = True)

In [41]:
train_lst = []
for i in glob.glob('../data/train_new_bioes/*.txt'):
    tmp =utils.read_txt(i)
    train_lst.append([(i.split(' ')[0],i.split(' ')[1].split('\n')[0]) for i in tmp])
val_lst = []
for i in glob.glob('../data/val_new_bioes/*.txt'):
    tmp =utils.read_txt(i)
    val_lst.append([(i.split(' ')[0],i.split(' ')[1].split('\n')[0]) for i in tmp])

In [36]:
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for _, label in sent]

def sent2tokens(sent):
    return [token for ID, token, _ in sent]

In [39]:
def word2features(sent, i):
    word = sent[i][0]
    
    features = {
        'bias': 1.0,
        'word': word
    }
    
    if i > 0:
        word = sent[i-1][0]
        features.update({
            '-1:word': word
        })
    else:
        features['BOS'] = True

    if i > 1:
        word = sent[i-2][0]
        features.update({
            '-2:word': word
        })
    else:
        features['BOS'] = True   
                
        
    if i < len(sent)-1:
        word = sent[i+1][1]
        features.update({
            '+1:word': word
        })
    else:
        features['EOS'] = True

    if i < len(sent)-2:
        word = sent[i+2][1]
        features.update({
            '+2:word': word
        })
    else:
        features['EOS'] = True
        
    return features

In [44]:
%%time
X_train = [sent2features(s) for s in train_lst]
y_train = [sent2labels(s) for s in train_lst]

X_test = [sent2features(s) for s in val_lst]
y_test = [sent2labels(s) for s in val_lst]

CPU times: user 583 ms, sys: 38.8 ms, total: 622 ms
Wall time: 622 ms


In [45]:
%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=200,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

CPU times: user 5min, sys: 1 s, total: 5min 1s
Wall time: 5min 2s
